In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from models.cnn import SimpleCNN, LeNet

In [2]:
in_channels = 1
num_classes = 10
num_conv_layers = 2
temperature = 1
num_epochs = 10
batch_size = 64
lr = 0.001
save_path_folder = 'saved_models/model'

In [ ]:
#Load CNN
model = SimpleCNN(in_channels=in_channels, num_classes=num_classes, num_conv_layers=num_conv_layers, temperature=temperature).to('cuda:0')
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [3]:
# MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# model training
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images.to('cuda'))
        loss = criterion(outputs, labels.to('cuda'))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
# Save the trained model
torch.save(model.state_dict(), save_path_folder)
print(f"Model saved as {save_path_folder}!")

In [ ]:
torch.save(model.state_dict(), 'saved_models/model')

In [ ]:
state_dict = torch.load(save_path_folder)
model = SimpleCNN(in_channels=in_channels, num_classes=num_classes, num_conv_layers=num_conv_layers, temperature=temperature).to('cuda:0')
model.load_state_dict(state_dict=state_dict)

In [ ]:
# Testing the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images.to('cuda'))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to('cuda')).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f}')

In [ ]:
# MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)), transforms.Lambda(lambda x: x.view(-1))])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
from models.mlp import MLP
mlp = MLP(input_dim = 784, output_dim= num_classes, hidden_size= 2048, hidden_layers= 4, device='cuda')

Not using softmax


In [ ]:
for (im,lab) in train_loader:
    print(im.shape)

In [12]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=lr)
mlp.train(train_loader=train_loader, optimizer=optimizer, criterion=criterion, num_epochs=10)

Epoch [1/10], Step [100/938], Loss: 0.5841
Epoch [1/10], Step [200/938], Loss: 0.5482
Epoch [1/10], Step [300/938], Loss: 0.3348
Epoch [1/10], Step [400/938], Loss: 0.3186
Epoch [1/10], Step [500/938], Loss: 0.3854
Epoch [1/10], Step [600/938], Loss: 0.1683
Epoch [1/10], Step [700/938], Loss: 0.2387
Epoch [1/10], Step [800/938], Loss: 0.2411
Epoch [1/10], Step [900/938], Loss: 0.2360
Epoch [2/10], Step [100/938], Loss: 0.0225
Epoch [2/10], Step [200/938], Loss: 0.1381
Epoch [2/10], Step [300/938], Loss: 0.3077
Epoch [2/10], Step [400/938], Loss: 0.0668
Epoch [2/10], Step [500/938], Loss: 0.1085
Epoch [2/10], Step [600/938], Loss: 0.2506
Epoch [2/10], Step [700/938], Loss: 0.3483
Epoch [2/10], Step [800/938], Loss: 0.0506
Epoch [2/10], Step [900/938], Loss: 0.2477
Epoch [3/10], Step [100/938], Loss: 0.2222
Epoch [3/10], Step [200/938], Loss: 0.2445
Epoch [3/10], Step [300/938], Loss: 0.1772
Epoch [3/10], Step [400/938], Loss: 0.0625
Epoch [3/10], Step [500/938], Loss: 0.2393
Epoch [3/10

In [13]:
mlp.eval(test_loader=test_loader)

a
Test Accuracy: 0.9758


In [ ]:
from models.mlp import MLP
mlp = MLP(input_dim = 784, output_dim= num_classes, hidden_size= 2048, hidden_layers= 4, device='cuda', from_saved_state_dict="saved_models/mlp")
mlp.eval(test_loader=test_loader)